In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from gensim import corpora, models, matutils
import numpy as np
import ast

In [2]:
df_doc = pd.read_csv('datasets/10_tfidf.csv')
df_doc.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'claim_id', 'document_content',
       'similarity_score', 'claim_label', 'claim_content', 'document_bow_19.5',
       'document_tfidf_19.5', 'document_bow_19.0',
       ...
       'claim_bow_0.1', 'claim_tfidf_0.1', 'claim_bow_0.09',
       'claim_tfidf_0.09', 'claim_bow_0.08', 'claim_tfidf_0.08',
       'claim_bow_0.07', 'claim_tfidf_0.07', 'claim_bow_0.06',
       'claim_tfidf_0.06'],
      dtype='object', length=111)

In [3]:
for nb_word in sorted(list(np.arange(500,20000,500)),reverse=True):
    df_doc['document_bow_'+str(nb_word/1000)] = df_doc['document_bow_'+str(nb_word/1000)].apply(lambda st: ast.literal_eval(st))
    df_doc['document_tfidf_'+str(nb_word/1000)] = df_doc['document_tfidf_'+str(nb_word/1000)].apply(lambda st: ast.literal_eval(st))
for nb_word in sorted(list(np.arange(60,186,10)),reverse=True):    
    df_doc['claim_bow_'+str(nb_word/1000)] = df_doc['claim_bow_'+str(nb_word/1000)].apply(lambda st: ast.literal_eval(st))
    df_doc['claim_tfidf_'+str(nb_word/1000)] = df_doc['claim_tfidf_'+str(nb_word/1000)].apply(lambda st: ast.literal_eval(st))

In [4]:
for nb_word in sorted(list(np.arange(500,20000,500)),reverse=True):
    for nb_topic in np.arange(20,310,20):
        lsi = models.LsiModel.load('topic_models/10_doc_'+str(nb_word/1000)+'_'+str(nb_topic)+'.lsi')
        rp = models.RpModel.load('topic_models/10_doc_'+str(nb_word/1000)+'_'+str(nb_topic)+'.rp')
        lda = models.RpModel.load('topic_models/10_doc_'+str(nb_word/1000)+'_'+str(nb_topic)+'.lda')
        
        df_doc['document_lsi_'+str(nb_word/1000)+'_'+str(nb_topic)] = \
        df_doc['document_tfidf_'+str(nb_word/1000)].apply(lambda x: lsi[x])
        df_doc['document_rp_'+str(nb_word/1000)+'_'+str(nb_topic)] = \
        df_doc['document_bow_'+str(nb_word/1000)].apply(lambda x: rp[x])
        df_doc['document_lda_'+str(nb_word/1000)+'_'+str(nb_topic)] = \
        df_doc['document_bow_'+str(nb_word/1000)].apply(lambda x: lda[x])

for nb_word in sorted(list(np.arange(60,186,10)),reverse=True):
    for nb_topic in np.arange(10,60,10):
        lsi = models.LsiModel.load('topic_models/10_claim_'+str(nb_word/1000)+'_'+str(nb_topic)+'.lsi')
        rp = models.RpModel.load('topic_models/10_claim_'+str(nb_word/1000)+'_'+str(nb_topic)+'.rp')
        lda = models.RpModel.load('topic_models/10_claim_'+str(nb_word/1000)+'_'+str(nb_topic)+'.lda')
        
        df_doc['claim_lsi_'+str(nb_word/1000)+'_'+str(nb_topic)] = \
        df_doc['claim_tfidf_'+str(nb_word/1000)].apply(lambda x: lsi[x])
        df_doc['claim_rp_'+str(nb_word/1000)+'_'+str(nb_topic)] = \
        df_doc['claim_bow_'+str(nb_word/1000)].apply(lambda x: rp[x])
        df_doc['claim_lda_'+str(nb_word/1000)+'_'+str(nb_topic)] = \
        df_doc['claim_bow_'+str(nb_word/1000)].apply(lambda x: lda[x])

In [5]:
feature_categories_document = []
for nb_word in sorted(list(np.arange(500,20000,500)),reverse=True):
    for f in ['bow','tfidf']:
        feature_categories_document.append('document_'+f+'_'+str(nb_word/1000))
    for nb_topic in np.arange(20,310,20):
        for f in ['lsi','lda','rp']:
            feature_categories_document.append('document_'+f+'_'+str(nb_word/1000)+'_'+str(nb_topic))
feature_categories_claim = []            
for nb_word in sorted(list(np.arange(60,186,10)),reverse=True):
    for f in ['bow','tfidf']:
        feature_categories_claim.append('claim_'+f+'_'+str(nb_word/1000))
    for nb_topic in np.arange(10,60,10):
        for f in ['lsi','lda','rp']:
            feature_categories_claim.append('claim_'+f+'_'+str(nb_word/1000)+'_'+str(nb_topic))

In [7]:
df_doc[['claim_id','claim_label']+feature_categories_document+feature_categories_claim].to_csv('datasets/11_features.csv')